In [ ]:
! pip install kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
#https://www.kaggle.com/code/bettycxh06/rri-amplitudes/input
!kaggle datasets download -d bettycxh06/apnea-ecg-database

!kaggle kernels pull bettycxh06/rri-amplitudes


100% 314M/315M [00:18<00:00, 21.4MB/s]
100% 315M/315M [00:18<00:00, 17.8MB/s]
Source code downloaded to /content/rri-amplitudes.ipynb


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
os.listdir(os.getcwd())

['.config',
 'kaggle.json',
 'apnea-ecg-database.zip',
 'rri-amplitudes.ipynb',
 'sample_data']

In [ ]:
!unzip apnea-ecg-database.zip

Archive:  apnea-ecg-database.zip
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/ANNOTATORS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/RECORDS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/SHA256SUMS.txt  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.dat  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01r.apn  

In [ ]:
!pip install biosppy
!pip install wfdb

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import posixpath
import csv
import wfdb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
import pickle
import sys
from concurrent.futures import ProcessPoolExecutor, as_completed

import biosppy.signals.tools as st
import numpy as np
import os
import wfdb
from biosppy.signals.ecg import correct_rpeaks, hamilton_segmenter
from scipy.signal import medfilt
from tqdm import tqdm

# PhysioNet Apnea-ECG dataset
# url: https://physionet.org/physiobank/database/apnea-ecg/
#base_dir = "../input/apnea-ecg-database/apnea-ecg-database-1.0.0"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"



fs = 100
sample = fs * 60  # 1 min's sample points

before = 2  # forward interval (min)
after = 2  # backward interval (min)
hr_min = 20
hr_max = 300

num_worker = 35


def worker(name, labels):
    X = []
    y = []
    groups = []
    signals = wfdb.rdrecord(os.path.join(base_dir, name), channels=[0]).p_signal[:, 0]
    for j in tqdm(range(len(labels)), desc=name, file=sys.stdout):
        if j < before or \
                (j + 1 + after) > len(signals) / float(sample):
            continue
        signal = signals[int((j - before) * sample):int((j + 1 + after) * sample)]
        signal, _, _ = st.filter_signal(signal, ftype='FIR', band='bandpass', order=int(0.3 * fs),
                                        frequency=[3, 45], sampling_rate=fs)
        # Find R peaks
        rpeaks, = hamilton_segmenter(signal, sampling_rate=fs)
        rpeaks, = correct_rpeaks(signal, rpeaks=rpeaks, sampling_rate=fs, tol=0.1)
        if len(rpeaks) / (1 + after + before) < 40 or \
                len(rpeaks) / (1 + after + before) > 200:  # Remove abnormal R peaks signal
            continue
        # Extract RRI, Ampl signal
        rri_tm, rri_signal = rpeaks[1:] / float(fs), np.diff(rpeaks) / float(fs)
        rri_signal = medfilt(rri_signal, kernel_size=3)
        ampl_tm, ampl_siganl = rpeaks / float(fs), signal[rpeaks]
        hr = 60 / rri_signal
        # Remove physiologically impossible HR signal
        if np.all(np.logical_and(hr >= hr_min, hr <= hr_max)):
            # Save extracted signal
            X.append([(rri_tm, rri_signal), (ampl_tm, ampl_siganl)])
            y.append(0. if labels[j] == 'N' else 1.)
            groups.append(name)
    return X, y, groups


if __name__ == "__main__":
    apnea_ecg = {}

    names = [
        "a01", "a02", "a03", "a04", "a05", "a06", "a07", "a08", "a09", "a10",
        "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20",
        "b01", "b02", "b03", "b04", "b05",
        "c01", "c02", "c03", "c04", "c05", "c06", "c07", "c08", "c09", "c10"
    ]

    o_train = []
    y_train = []
    groups_train = []
    print('Training...')
    with ProcessPoolExecutor(max_workers=num_worker) as executor:
        task_list = []
        for i in range(len(names)):
            labels = wfdb.rdann(os.path.join(base_dir, names[i]), extension="apn").symbol
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_train.extend(X)
            y_train.extend(y)
            groups_train.extend(groups)

    print()



    answers = {}
    with open(os.path.join("event-2-answers"), "r") as f:
    #    with open(os.path.join(base_dir, "event-2-answers"), "r") as f:
        for answer in f.read().split("\n\n"):
            answers[answer[:3]] = list("".join(answer.split()[2::2]))

    names = [
        "x01", "x02", "x03", "x04", "x05", "x06", "x07", "x08", "x09", "x10",
        "x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20",
        "x21", "x22", "x23", "x24", "x25", "x26", "x27", "x28", "x29", "x30",
        "x31", "x32", "x33", "x34", "x35"
    ]

    o_test = []
    y_test = []
    groups_test = []
    print("Testing...")
    with ProcessPoolExecutor(max_workers=num_worker) as executor:

        task_list = []
        for i in range(len(names)):
            labels = answers[names[i]]
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_test.extend(X)
            y_test.extend(y)
            groups_test.extend(groups)

    apnea_ecg = dict(o_train=o_train, y_train=y_train, groups_train=groups_train, o_test=o_test, y_test=y_test,
                     groups_test=groups_test)
    with open(os.path.join(base_dir, "apnea-ecg.pkl"), "wb") as f:
        pickle.dump(apnea_ecg, f, protocol=2)

    print("\nok!")

Training...
a17: 100%|██████████| 485/485 [45:00<00:00,  5.57s/it]

Testing...
x12: 100%|██████████| 527/527 [46:41<00:00,  5.32s/it]

ok!


In [ ]:
!pip install keras
!pip install tensorflow
#
#IMPORT LIBRARIES
import pickle
import numpy as np
import tensorflow
import os
from keras.callbacks import LearningRateScheduler,EarlyStopping
from keras.layers import  Dense,Flatten,MaxPooling2D,Conv2D,Permute,Reshape,LSTM,BatchNormalization,Bidirectional
from keras.regularizers import l2
from scipy.interpolate import splev, splrep
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
import keras
import matplotlib.pyplot as plt
from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D,Conv2D
from keras.models import Model, load_model, save_model
from keras.layers import Input
import pandas as pd
from keras.layers import GRU
from keras.layers import add, concatenate
from keras.layers import Reshape, Lambda

#base_dir = "dataset"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"

#------------------------------------------------------------------------------
# DEEP LEARNING MODELS NEED TO VECTORS OR MATRIX WITH SAME SIZE
# R-R INTERVALS DONT HAVE SAME SIZE, SO WE NEED TO INTERPOLATE VECTORS TO GET VECTORS WITH SAME SIZE.
# BASED ON OUR EXPERIENCE INTERPOLATION IN 3 HZ BETTER AND ACCURATE.
ir = 3 # INTERPOLATION RATE(3HZ)
time_range= 60 # 60-s INTERVALS OF ECG SIGNALS
weight=1e-3 #  WEIGHT L2 FOR REGULARIZATION(AVODING OVERFITTING PARAMETER)
#-----------------------------
# NORMALIZATION:
# DEEP LEARNING AND EVEN NEURAL NETWORKS INPUT SHOULD BE NORMALIZED:
# MIN-MAX METHOD APPLIED FOR SCALING:(Array-min(Array))/(max(Array)-min(Array))
scaler = lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr))
#-----------------------------
# FIRSTLY WE PRE-PROCESSED OUR DATA IN "apnea-ecg.pkl" FILE
# IN PRE-PROCESSING SECTION WE EXTRACT R-R INTERVALS AND R-PEAK AMPLITUDES
# IN THIS PART WE LOAD THIS DATA AND INTERPOLATE AND CONCATE FOR FEEDING TO NETWORKS
def load_data():
    tm = np.arange(0, (before + 1 + after) * 60, step=1 / float(ir))

    with open(os.path.join(base_dir, "apnea-ecg.pkl"), 'rb') as f: # read preprocessing result
        apnea_ecg = pickle.load(f)
#-----------------
    x_train = []
    o_train, y_train = apnea_ecg["o_train"], apnea_ecg["y_train"]

    for i in range(len(o_train)):
        (rri_tm, rri_signal), (amp_tm, amp_signal) = o_train[i]
      # Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        amp_interp_signal = splev(tm, splrep(amp_tm, scaler(amp_signal), k=3), ext=1)
        x_train.append([rri_interp_signal, amp_interp_signal])
    x_train = np.array(x_train, dtype="float32")

    x_train = np.expand_dims(x_train,1)
    x_train=np.array(x_train, dtype="float32").transpose((0,3,1,2)) # convert to numpy format

    #return x_train_final, y_train
    return x_train, y_train

In [ ]:
!pip install Hyperactive
!pip install mealpy

from keras.layers.rnn import GRU
from keras.layers import Reshape, Lambda
from keras.layers import GRU
from hyperactive import Hyperactive
from hyperactive.optimizers import HillClimbingOptimizer
from tensorflow.keras import optimizers
#from Hyperactive.hyperactive import RandomSearchOptimizer, MHoneyBadgerAlgorithm, HoneyBadgerAlgorithm
from keras.optimizers import adam
import threading
import concurrent.futures
from threading import Thread
import time
from tabnanny import verbose
from mealpy.swarm_based.HBA import OriginalHBA
from tensorflow.python.keras import optimizers
from keras.optimizers import adam
#from HBA import HBA
import warnings
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.0/378.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 46.6 MB/s eta 0:00:00


In [ ]:
#print(x_train.shape, y_train.shape)
#x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
#x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
#x_train= x_train.transpose((0,2,1))
#print(x_train.shape, y_train.shape)
##(16709, 900, 1, 2) (16709, 2)
##(16709, 2, 900) (16709, 2)


In [ ]:
# BHA
###########################
def fitness_function(position):
    learning_rate,n1,n2,n3,n4,n5,n6,n7,epochs = position[0],position[1],position[2],position[3],position[4],position[5],position[6],position[7],position[8]
    n1,n2,n3,n4,n5,n6,n7,epochs=int(n1),int(n2),int(n3),int(n4),int(n5),int(n6),int(n7),int(epochs)
    #return learning_rate,n1,n2,n3,n4,n5,n6,epochs

    print(learning_rate,n1,n2,n3,n4,n5,n6,n7,epochs)

#def create_model(weight=1e-3):
    model=Sequential()
    model.add(Reshape((450,2,2),input_shape=(900,1,2)))
    model.add(Conv2D(n1, kernel_size=(11,1), strides=(1,1), padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight),input_shape=(900,1,2)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,1)))
    model.add(Conv2D(n2, kernel_size=(5,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,1)))
    model.add(Conv2D(n3, kernel_size=(3,1), strides=(1,1), padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(Conv2D(n4, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(Conv2D(n5, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,1),strides=(2,1)))

    model.add(Permute((2,1,3)))
    model.add(Reshape((2,24*n5)))

    model.add(LSTM(n6, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(n7, activation="relu"))

    model.add(Dense(2, activation="softmax"))

    return learning_rate,n1,n2,n3,n4,n5,n6,n7,epochs

    optimizer =keras.optimizers.Adam(lr=learning_rate)
    print("Learning rate: ", learning_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    #model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=['accuracy'])

    history=model.fit(X1, Y1, batch_size=128, epochs=int(epochs), validation_data=(x_val, y_val),
                       callbacks=[callback1,lr_scheduler])


    #history = model.fit(x_train,y_train, epochs=int(epochs),batch_size=32, validation_data=(x_test, y_test))
    return history.history['val_accuracy'][-1]
    #loss, accuracy = model.evaluate(x_test, y_test)

    #return model
#------------------------------------------------------------------------------
# Define learning rate schedule for preventing overfitting in deep learning methods:
def lr_schedule(epochs, learning_rate):
   if epochs > 70 and \
           (epochs - 1) % 10 == 0:
        learning_rate *= 0.1
   print("Learning rate: ", learning_rate)
   return learning_rate

#------------------------------------------------------------------------------

        # we used k-fold cross-validation for more reliable experiments:
   kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)
   cvscores = []
   ACC=[]
   SN=[]
   SP=[]
   F2=[]
   print("train num:", len(y_train))
   print("test num:", len(y_test))


    # separate train& test and then compile model
  # for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

     # Compile and evaluate model:
if __name__ == "__main__":
    # loading Data:
    x_train, y_train = load_data()
    # we have labels(Y) in a binary way 0 for normal and 1 for apnea patients
    # we want to classify data into 2-class so we changed y in a categorical way:
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
    # we used k-fold cross-validation for more reliable experiments:
    kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)


    # separate train& test and then compile model
    ##for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

    # define callback for early stopping:
    lr_scheduler = LearningRateScheduler(lr_schedule)
    callback1 = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    #x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
    #x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
    #x_train= x_train.transpose((0,2,1))
    #10% of Data used for validation:
    X1,x_val,Y1,y_val=train_test_split(x_train,y_train,test_size=0.10)

##################################################################
#pso = Pso(swarmsize=4,maxiter=14)
# n,sf,sp,l
##bp,value = pso.run(func,[1,2,2,2],[16,8,4,4])
#v = func(bp);
##################################################################
problem_dict1 = {
    "fit_func": fitness_function,
    'lb':[0.001,20,40,128,128,40,16,4,100], # Lower bound of our parameters
    'ub':[0.001,256,700,700,700,512,256,64,100], # upper bound of our parameters
    "minmax": "min",
    "obj_weights": [0.4, 1, 1,1,1,1,1,1,1]               # Define it or default value will be [1, 1, 1]   [0.4, 0.1, 0.5,1,1,1,1,1]
                }

rng = np.random.default_rng()
epoch = 100
pop_size =50

#def thread_function(k):

  #run=k+1
  #print("Run Number",run)
  #Optimizer = MHoneyBadgerAlgorithm(search_config, n_iter=100, n_part=10, metric='accuracy', cv=10, h_beta=6.0, h_c=2.0,run=run)
model = OriginalHBA(epoch, pop_size)
  #t1 = time.time()

  #Optimizer.fit(x_train, y_train)
 # t2 = time.time()

  #print("time: {}".format(t2-t1))



#best_position, best_fitness = model.solve(problem_dict1)
best_position, best_fitness= model.solve(problem_dict1)
#best_position, best_fitness= model.solve(mode="thread")
v = fitness_function(best_position);
#print(f"Solution: {best_position}, Fitness: {best_fitness}")
#return v
print(f"Solution: {best_position}, Fitness: {v}")

0.001 149 557 191 670 374 239 38 100


INFO:mealpy.swarm_based.HBA.OriginalHBA:Solving 9-objective optimization problem with weights: [0.4 1.  1.  1.  1.  1.  1.  1.  1. ].


0.001 174 301 406 141 61 135 45 100
0.001 88 510 659 664 148 216 13 100
0.001 120 277 377 342 193 45 54 100
0.001 147 54 668 644 389 253 55 100
0.001 102 308 661 242 84 93 52 100
0.001 239 170 256 562 376 120 63 100
0.001 136 632 260 393 407 95 61 100
0.001 206 239 487 388 140 139 49 100
0.001 237 99 368 466 445 164 46 100
0.001 40 210 655 477 232 168 28 100
0.001 241 645 440 186 64 157 60 100
0.001 140 642 568 585 300 92 49 100
0.001 54 162 278 260 493 114 49 100
0.001 77 594 217 495 495 52 15 100
0.001 74 97 595 481 206 92 25 100
0.001 156 695 264 446 66 59 38 100
0.001 192 402 643 656 199 75 44 100
0.001 223 449 550 142 408 225 57 100
0.001 135 467 437 520 319 65 20 100
0.001 28 348 603 228 270 200 43 100
0.001 41 637 486 334 452 160 28 100
0.001 159 255 562 401 254 202 51 100
0.001 239 515 638 145 252 100 52 100
0.001 59 306 132 541 84 217 37 100
0.001 125 532 581 624 340 45 7 100
0.001 223 174 567 224 142 84 60 100
0.001 147 427 512 350 125 193 30 100
0.001 170 264 584 496 211 211

INFO:mealpy.swarm_based.HBA.OriginalHBA:>Problem: P, Epoch: 1, Current best: 697.0004, Global best: 697.0004, Runtime: 22.73722 seconds


0.001 256 700 128 700 165 31 64 100
0.001 20 40 128 150 190 16 4 100
0.001 51 170 245 173 165 35 7 100
0.001 20 42 129 199 176 32 4 100
0.001 39 137 188 215 85 34 26 100
0.001 20 40 128 252 136 31 4 100
0.001 21 40 128 206 132 20 4 100
0.001 68 119 128 187 157 30 7 100
0.001 23 40 128 230 188 57 4 100
0.001 20 40 171 254 148 29 4 100
0.001 137 153 169 215 146 33 5 100
0.001 41 40 128 205 167 33 4 100
0.001 20 40 128 200 40 27 4 100
0.001 20 47 147 128 190 28 4 100
0.001 20 50 136 199 172 39 4 100
0.001 20 58 128 191 151 29 4 100
0.001 20 56 128 133 165 30 4 100
0.001 20 40 128 210 105 16 4 100
0.001 63 123 128 198 159 40 32 100
0.001 20 48 128 176 183 40 4 100
0.001 25 55 150 205 143 21 35 100
0.001 20 40 128 192 151 16 4 100
0.001 23 139 317 187 128 46 9 100
0.001 20 49 138 157 40 40 4 100
0.001 20 48 128 218 183 34 4 100
0.001 24 40 158 243 159 27 4 100
0.001 20 50 142 128 40 30 24 100
0.001 68 109 155 205 163 32 5 100
0.001 20 40 128 197 178 32 4 100
0.001 20 51 128 128 180 16 4 100

...

0.001 20 41 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 256 700 700 700 512 16 64 100
0.001 20 40 128 128 40 16 4 100
0.001 20 47 128 128 40 18 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 41 128 128 46 16 4 100
0.001 256 40 700 700 512 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 43 40 128 128 40 35 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 41 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 22 46 128 144 45 16 4 100
0.001 22 46 146 128 45 18 4 100
0.001 77 40 497 497 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.

INFO:mealpy.swarm_based.HBA.OriginalHBA:>Problem: P, Epoch: 99, Current best: 476.0004, Global best: 476.0004, Runtime: 22.72666 seconds


0.001 20 40 128 128 40 16 4 100
0.001 23 47 128 128 46 16 4 100
0.001 20 40 700 700 512 256 4 100
0.001 20 40 128 128 40 16 4 100
0.001 31 41 204 128 40 24 6 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 361 144 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 46 128 128 40 16 4 100
0.001 20 40 129 129 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 41 128 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 42 132 128 40 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 26 41 170 170 53 16 5 100
0.001 26 41 172 128 40 16 5 100
0.001 21 43 135 128 42 16 4 100
0.001 20 46 128 146 40 18 4 100
0.001 20 41 128 128 40 16 4 100
0.001 20 41 166 166 52 16 4 100
0.001 20 40 128 128 40 16 4 100
0.001 20 41 139 128 43 16 4 100
0.001 20 40 128 140 43 17 4 100
0.001 20 174 128 548 171 68 17 100
0.001 20 40 128 128 40 16 4 100
0.001 20 653 128 128 512 255 4 100
0.001 21 43 136 136 40 17 4 100
0.001 20 40 128 128 40 16 4 10

INFO:mealpy.swarm_based.HBA.OriginalHBA:>Problem: P, Epoch: 100, Current best: 476.0004, Global best: 476.0004, Runtime: 21.91364 seconds


0.001 20 40 128 128 40 16 4 100
Solution: [1.00000000e-03 2.00000000e+01 4.08539541e+01 1.28000000e+02
 1.28000000e+02 4.00000000e+01 1.60000000e+01 4.00000000e+00
 1.00000000e+02], Fitness: (0.001, 20, 40, 128, 128, 40, 16, 4, 100)
